In [1]:
# import splinter and BeatifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from quant_up import user, password
import time 
import datetime
import pandas as pd

In [ ]:
### Navigate to quantum online,input credentials and navigate to securities for redemption 

In [2]:
# confirm todays date
datetime.datetime.today().strftime("%Y-%m-%d")

'2023-03-16'

In [3]:
# redemptions page reoutes to login, 
browser = Browser('chrome')

In [4]:
# Visit quantum https://www.quantumonline.com/stklist.cfm?type=Redemptions&RequestTimeout=60
url = 'https://www.quantumonline.com/stklist.cfm?type=Redemptions&RequestTimeout=60'
browser.visit(url)

In [6]:
browser.quit()